# Trabalho Prático

## Objetivos de Ensino

Neste trabalho, você vai exercitar os conceitos trabalhados na primeira parte do módulo, e vai:

✔ Se acostumar a escrever e executar aplicações que usam o Spark;

✔ Construir aplicações Spark interativas usando o pyspark ou uma plataforma interativa como o jupyter-lab;

✔ Computar estatísticas descritivas usando o Spark;

✔ Manipular dados a partir da API de DataFrames.


# Enunciado

Dados do mercado financeiro são interessantes e ricos: cada ação negociada na bolsa de valores tem um preço que varia a cada dia. Você foi contratado como cientista de dados de uma empresa de Wall Street para criar modelos preditivos que, a partir da variação diária do preço das ações, consigam subsidiar e melhorar decisões de compra e venda de ações. Você disse que, como todo bom cientista de dados, gostaria de explorar os dados para entender suas características antes de criar qualquer modelo preditivo.

Os dados estão disponíveis em https://www.kaggle.com/camnugent/sandp500/ por meio do arquivo all_stocks_5yr.csv. O arquivo contém, para cada dia e ação do S&P 500 (lista de 500 maiores empresas americanas), os seguintes dados:

● Date - no formato yy-mm-dd

● Open - Preço da ação na abertura do mercado no dia, em dólares.

● High - Maior preço alcançado naquele dia

● Low - Menor preço alcançado naquele dia

● Close - Preço da ação no fechamento do mercado no dia

● Volume - Número de ações vendidas / compradas

● Name - O nome da ação.

Apesar do volume de dados ser pequeno, você decidiu usar o Apache Spark para processar os dados para aprender a ferramenta, e tendo em vista que a sua empresa disse que, em breve, obterá dados por minuto, e não por dia, e de todas as ações do planeta, não apenas dos Estados Unidos. Neste caso, uma ferramenta desenhada para lidar com big data será necessária, e você já quer estar com o código pronto.

In [1]:
from pyspark.sql import SparkSession

# Spark entry point
spark = SparkSession \
    .builder \
    .appName("Atividade pratica 2 - Cientista de Dados - Apache Spark") \
    .getOrCreate()

In [2]:
spark.version

'3.3.1'

In [3]:
# Carregando a planilha CSV

mercado_df = spark.read.csv('all_stocks_5yr.csv',header='True',inferSchema='True')

In [4]:
# Detalhando os atributos do data frame
mercado_df.printSchema()

root
 |-- date: timestamp (nullable = true)
 |-- open: double (nullable = true)
 |-- high: double (nullable = true)
 |-- low: double (nullable = true)
 |-- close: double (nullable = true)
 |-- volume: integer (nullable = true)
 |-- Name: string (nullable = true)



In [18]:
# Visualizando os dados do df

mercado_df.show()

+-------------------+-----+-----+-----+-----+--------+----+
|               date| open| high|  low|close|  volume|Name|
+-------------------+-----+-----+-----+-----+--------+----+
|2013-02-08 00:00:00|15.07|15.12|14.63|14.75| 8407500| AAL|
|2013-02-11 00:00:00|14.89|15.01|14.26|14.46| 8882000| AAL|
|2013-02-12 00:00:00|14.45|14.51| 14.1|14.27| 8126000| AAL|
|2013-02-13 00:00:00| 14.3|14.94|14.25|14.66|10259500| AAL|
|2013-02-14 00:00:00|14.94|14.96|13.16|13.99|31879900| AAL|
|2013-02-15 00:00:00|13.93|14.61|13.93| 14.5|15628000| AAL|
|2013-02-19 00:00:00|14.33|14.56|14.08|14.26|11354400| AAL|
|2013-02-20 00:00:00|14.17|14.26|13.15|13.33|14725200| AAL|
|2013-02-21 00:00:00|13.62|13.95| 12.9|13.37|11922100| AAL|
|2013-02-22 00:00:00|13.57| 13.6|13.21|13.57| 6071400| AAL|
|2013-02-25 00:00:00| 13.6|13.76| 13.0|13.02| 7186400| AAL|
|2013-02-26 00:00:00|13.14|13.42| 12.7|13.26| 9419000| AAL|
|2013-02-27 00:00:00|13.28|13.62|13.18|13.41| 7390500| AAL|
|2013-02-28 00:00:00|13.49|13.63|13.39|1

### 01. Quantos registros há na planilha?

In [6]:
  # extraindo quantidade de linhas
  row = mercado_df.count()
   
  # extraindo quantidade de colunas
  col = len(mercado_df.columns)
 
  # print
  print(f'A dimensão da planilha é: {(row,col)}')
  print(f'Quantidade de linhas: {row}')
  print(f'Quantidade de colunas: {col}')

A dimensão da planilha é: (619040, 7)
Quantidade de linhas: 619040
Quantidade de colunas: 7


### 02. Quantos registros há na planilha para a ação da Apple (AAPL)?

In [7]:
totalAAPL = mercado_df.filter(mercado_df["Name"] == "AAPL").count()

print(f'Quantidade de registros para a ação da Apple (AAPL): {totalAAPL}')


Quantidade de registros para a ação da Apple (AAPL): 1259


### 03. Quantas empresas distintas têm registros nessa planilha?

In [8]:
totalEmp = mercado_df.select("Name").distinct().count()

print(f'A quantidade de empresas distintas presentes na planilha é: {totalEmp}')

A quantidade de empresas distintas presentes na planilha é: 505


### 04.  Com qual frequência o preço de uma ação no fechamento é maior do que o preço na abertura?

In [30]:
freqAcao = mercado_df.filter(mercado_df["close"] > mercado_df["open"])

freqAcao = (freqAcao.count() /mercado_df.count()) * 100

print(f'Frequência do preço de uma ação ser maior que na abertura: {round((freqAcao),2)}%')

Frequência do preço de uma ação ser maior que na abertura: 51.53%


### 5. Qual o maior valor das ações da Apple (AAPL) na história?

In [11]:
from pyspark.sql.functions import max

Apple = mercado_df.filter(mercado_df["Name"] == "AAPL").agg(max("high")).first()[0]

print(f'O maior valor das ações da Apple é: {Apple}')

O maior valor das ações da Apple é: 180.1


### 6. Qual ação tem a maior volatilidade? Uma forma é medir o desvio-padrão do preço de fechamento de cada ação e considerar a ação de maior desvio-padrão.

In [12]:
from pyspark.sql.functions import stddev

AcaoVol = mercado_df.groupBy("Name").agg(stddev("close").alias("std"))
AcaoVol = AcaoVol.select("Name", "std").orderBy("std", ascending=False)

print(f'Ação com maior volatilidade é: {AcaoVol.first()["Name"]}')

Ação com maior volatilidade é: PCLN


### 07. Qual o dia com maior volume total de negociação da bolsa?

In [13]:
from pyspark.sql.functions import sum

dia = mercado_df.groupBy("date").agg(sum("volume").alias("Maior volume total"))
dia = dia.select("date", "Maior volume total").orderBy("Maior volume total", ascending=False)

print(f'O dia com maior volume total de negociação na bolsa é: {dia.first()["date"]}')


O dia com maior volume total de negociação na bolsa é: 2015-08-24 00:00:00


### 08. Qual a ação mais negociada da bolsa, em volume de transações?

In [14]:
negociado = mercado_df.groupBy("Name").agg(sum("volume").alias("Ação mais negociada"))
negociado = negociado.select("Name","Ação mais negociada").orderBy("Ação mais negociada", ascending=False)

print(f'A ação mais negociada na bolsa é: {negociado.first()["Name"]}')


A ação mais negociada na bolsa é: BAC


### 09. Quantas ações começam com a letra “A”?

In [26]:

letraA = mercado_df.select("Name").distinct()
letraA = letraA.filter("Name like 'A%'")

print(f'Quantidade de ações que começam com a letra A é: {letraA.count()}')


Quantidade de ações que começam com a letra A é: 59


### 10. Com qual frequência o preço mais alto do dia da ação também é o preço de fechamento?

In [29]:
freqPreco = mercado_df.filter(mercado_df["high"] == mercado_df["close"])
freqPreco = (freqPreco.count() / mercado_df.count()) * 100

print(f'A frequência do preço mais alto do dia ser também o preço de fechamento é: {round((freqPreco),2)}%')

A frequência do preço mais alto do dia ser também o preço de fechamento é: 1.2%


### 11. Em qual dia a ação da Apple mais subiu entre a abertura e o fechamento, de forma absoluta?

In [33]:
from pyspark.sql.functions import abs

apple = mercado_df.filter(mercado_df["Name"] == "AAPL")
apple = apple.withColumn("Valor absoluto", abs(apple["close"] - apple["open"]))
apple = apple.select("date", "Valor absoluto").orderBy("Valor absoluto", ascending=False)

print(f'Dia em que a ação da Apple mais subiu entre a abertura e o fechamento: {apple.first()["date"]}')


Dia em que a ação da Apple mais subiu entre a abertura e o fechamento: 2015-08-24 00:00:00


### 12. Em média, qual o volume diário de transações das ações da AAPL?

In [36]:
from pyspark.sql.functions import avg

mediaApple = mercado_df.filter(mercado_df["Name"] == "AAPL")
mediaApple = mediaApple.agg(avg("volume")).first()[0]

print(f'A média do volume diário de transações das ações da AAPL é: {mediaApple}') 

A média do volume diário de transações das ações da AAPL é: 54047899.73550437


### 13. Quantas ações tem 1, 2, 3, 4 e 5 caracteres em seu nome, respectivamente?

In [39]:
from pyspark.sql.functions import length
from pyspark.sql.functions import col

nomeDistintos = mercado_df.select("Name").distinct()

nome = nomeDistintos.withColumn("Tamanho", length(col("Name")))
nome1 = nome.filter(nome["Tamanho"] == 1).count()
nome2 = nome.filter(nome["Tamanho"] == 2).count()
nome3 = nome.filter(nome["Tamanho"] == 3).count()
nome4 = nome.filter(nome["Tamanho"] == 4).count()
nome5 = nome.filter(nome["Tamanho"] == 5).count()

print(f'Quantidade de ações que possuem 1, 2, 3, 4 e 5 caracteres em seu nome, respectivamente, são: {nome1}, {nome2}, {nome3}, {nome4}, {nome5}')

Quantidade de ações que possuem 1, 2, 3, 4 e 5 caracteres em seu nome, respectivamente, são: 10, 50, 323, 117, 5


### 14. Qual a ação menos negociada da bolsa, em volume de transações?

In [40]:
volumeMenor = mercado_df.groupBy("Name").agg(sum("volume").alias("Total Volume"))
volumeMenor = volumeMenor.select("Name", "Total Volume").orderBy("Total Volume", ascending=True)

print(f'A ação menos negociada da bolsa é: {volumeMenor.first()["Name"]}')

A ação menos negociada da bolsa é: APTV
